In [1]:
import pandas as pd
from pathlib import Path
import itertools

In [2]:
df = pd.read_excel(Path.cwd() / "input_data" / "SProf94_final_substanceList.xlsx", sheet_name="substances")
df = df[df['geo'].isnull()]
df

,name,unit,cas,description,sumi,mainCompartment,geo,baseName
0,Actinium,kg,007440-34-8,Formula: Ac,90004354-71D3-47E8-B322-300BA5A98F7B,Raw materials,NaN,Actinium
1,Air,kg,132259-10-0,NaN,09336180-36F4-4389-9F51-1548DF956B24,Raw materials,NaN,Air
2,Aluminium,kg,007429-90-5,Formula: Al,0CDDB90F-BD92-46DE-BBDF-B7244C502CEB,Raw materials,NaN,Aluminium
3,Anhydrite,kg,014798-04-0,Formula: CaSO4,9F3A82FA-C5EC-4A67-A6BA-A8C70B506729,Raw materials,NaN,Anhydrite
4,Animal matter,kg,NaN,NaN,3AEDBE91-F73E-4283-ABDC-F5EFD119E784,Raw materials,NaN,Animal matter
...,...,...,...,...,...,...,...,...
24251,Compensation of employees incl. social contrib...,MEUR2011,NaN,NaN,1774FA7B-4603-420C-B5D7-A83EC47A5DE2,Economic issues,NaN,Compensation of employees incl. social contrib...
24252,Operating surplus: Consumption of fixed capital,MEUR2011,NaN,NaN,59B51226-9703-4DD5-B060-833C46CA2CDC,Economic issues,NaN,Operating surplus: Consumption of fixed capital
24253,Operating surplus: Remaining net operating sur...,MEUR2011,NaN,NaN,8B6D32E8-4257-44E0-A10E-9819CFB8A870,Economic issues,NaN,Operating surplus: Remaining net operating sur...
24254,Other net taxes on production,MEUR2011,NaN,NaN,9B212C13-1EEE-41ED-A6E9-19A10953787F,Economic issues,NaN,Other net taxes on production


The `FlowList` shoud follow the [standard template](https://github.com/UNEP-Economy-Division/GLAD-ElementaryFlowResources/blob/master/Formats/FlowList.md):

 Field | Type | Required |  Note |
----------- |  ---- | ---------| -----  |
 Flowable | string | Y | The flow name |
 CAS No | string | N | CAS number |
 Formula | string | N | Chemical formula|
 Synonyms | string | N | Flow synonyms
 Unit | string | Y  | The reference unit. uses [olca-ipc.py](https://github.com/GreenDelta/olca-ipc.py) units |
 Class | string | Y | The flow class, e.g. `Energy` or `Chemical` |
 External Reference | string | N | E.g. a web link |
 Preferred | int |  N |   `1` for preferred*, `0` for non-preferred
 Context | string | Y | A path-like set of context compartments e.g. `emission/air`|  
 Flow UUID | string | Y | Unique hexadecimal ID for the flow |
 AltUnit | string | N | Alternate unit for the flow |
 AltUnitConversionFactor | float | N | Conversion factor in the form of alternate units/reference unit |

First, we can rename some columns:

In [3]:
df = df.rename(columns={"name": "Flowable", "cas": "CAS No", "unit": "Unit", "mainCompartment": "Context", "baseName": "Synonyms", "sumi": "Flow UUID", "description": "Description"})
df = df.drop("geo", axis=1)

The chemical formula can be stored in the description field:

In [4]:
df['Formula'] = pd.Series([(string.replace("Formula: ", "").strip() if "Formula: " in str(string) else None) for string in df['Description']])
df['Description'][df['Formula'].notnull()] = None

Add required `Class` column:

In [5]:
df['Class'] = df['Context']

In [6]:
df

,Flowable,Unit,CAS No,Description,Flow UUID,Context,Synonyms,Formula,Class
0,Actinium,kg,007440-34-8,None,90004354-71D3-47E8-B322-300BA5A98F7B,Raw materials,Actinium,Ac,Raw materials
1,Air,kg,132259-10-0,NaN,09336180-36F4-4389-9F51-1548DF956B24,Raw materials,Air,None,Raw materials
2,Aluminium,kg,007429-90-5,None,0CDDB90F-BD92-46DE-BBDF-B7244C502CEB,Raw materials,Aluminium,Al,Raw materials
3,Anhydrite,kg,014798-04-0,None,9F3A82FA-C5EC-4A67-A6BA-A8C70B506729,Raw materials,Anhydrite,CaSO4,Raw materials
4,Animal matter,kg,NaN,NaN,3AEDBE91-F73E-4283-ABDC-F5EFD119E784,Raw materials,Animal matter,None,Raw materials
...,...,...,...,...,...,...,...,...,...
24251,Compensation of employees incl. social contrib...,MEUR2011,NaN,NaN,1774FA7B-4603-420C-B5D7-A83EC47A5DE2,Economic issues,Compensation of employees incl. social contrib...,NaN,Economic issues
24252,Operating surplus: Consumption of fixed capital,MEUR2011,NaN,NaN,59B51226-9703-4DD5-B060-833C46CA2CDC,Economic issues,Operating surplus: Consumption of fixed capital,NaN,Economic issues
24253,Operating surplus: Remaining net operating sur...,MEUR2011,NaN,NaN,8B6D32E8-4257-44E0-A10E-9819CFB8A870,Economic issues,Operating surplus: Remaining net operating sur...,NaN,Economic issues
24254,Other net taxes on production,MEUR2011,NaN,NaN,9B212C13-1EEE-41ED-A6E9-19A10953787F,Economic issues,Other net taxes on production,NaN,Economic issues


In [7]:
df = df[['Flowable', 'CAS No', 'Formula', 'Synonyms', 'Unit', 'Class', 'Context', 'Flow UUID', 'Description']]

In [8]:
df.to_csv(Path.cwd() / "input_data" / "SimaProv9.4.csv", index=False)